# Load Google Drive & Switch to ML Root Folder

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "gdrive/MyDrive/ML"

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML


# Install & Build Pre-requisites for PlaneRCNN (not to be executed when using MidasNet / Yolo V3)

In [ ]:
!echo "Installing CUDA 8"
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!rm cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
!apt-get update
!apt-get install cuda-8-0
!pip install torch==0.4.1
!apt-get update && apt-get install build-essential software-properties-common -y
!add-apt-repository ppa:ubuntu-toolchain-r/test -y 
!apt-get update
!apt-get install gcc-5 g++-5 -y
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 50 --slave /usr/bin/g++ g++ /usr/bin/g++-5 
!apt-get install gcc-5 g++-5 g++-5-multilib gfortran-5
!update-alternatives --config gcc

Installing CUDA 8
--2020-11-20 14:11:19--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?9_uI5PVUpzc5T08jOlp2PXojIF5LScolwVZlkfIA8uuAmkW7sQWN2mIHavcxyqurZTeUixaXvx_F2Lp9MR3SvSzkIWbQmEJjs5eVVU3sLo1_X01ZNT7zowQ11zyCH2jdeuNAFUX52MXHn0qSNLhqrfVv2KzyufRoMkG__R4igrzLl-z1gtgbb9z14jTgljjLmMMxUMCvp74DRHZSiSF77KQlDg [following]
--2020-11-20 14:11:20--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?9_uI5PVUpzc5T08jOlp2PXojIF5LScolwVZlkfIA8uuAmkW7sQWN2mIHavcxyqurZTeUixaXvx_F2L

# Build PlaneRCNN (not to be executed when using MidasNet / Yolo V3)

In [ ]:
#!echo "Downloading planercnn"
%cd "/content/gdrive/My Drive/ML"
#!rm -rf planercnn
#!git clone https://github.com/NVlabs/planercnn
%cd planercnn/nms/src/cuda/
!echo "Compiling nms"
!nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_50
%cd ../../
!echo "building build.py"
!python build.py
%cd ..
%cd roialign/roi_align/src/cuda/
!echo "Compiling roi_align"
!nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_50
%cd ../../
!python build.py
%cd ../../
#!pip install torch==0.4.1
!pwd
#!rm -r checkpoint
#!mkdir checkpoint
#!ls "/content/gdrive/My Drive/ML/model/"
#!gsutil cp -r "/content/gdrive/My Drive/ML/model/planercnn_normal_warping_refine" "checkpoint/"
# !echo "Running Tests"
# !python evaluate.py --methods=f --numTestingImages=5588 --suffix=warping_refine --dataset=inference --customDataFolder="/content/gdrive/My Drive/ML/data/interiors"
# !echo "Setup Complete"

/content/gdrive/My Drive/ML
/content/gdrive/My Drive/ML/planercnn/nms/src/cuda
Compiling nms
/content/gdrive/My Drive/ML/planercnn/nms
building build.py
Including CUDA code.
/content/gdrive/My Drive/ML/planercnn/nms
generating /tmp/tmpfpi1c5ae/_nms.c
setting the current directory to '/tmp/tmpfpi1c5ae'
running build_ext
building '_nms' extension
creating content
creating content/gdrive
creating content/gdrive/My Drive
creating content/gdrive/My Drive/ML
creating content/gdrive/My Drive/ML/planercnn
creating content/gdrive/My Drive/ML/planercnn/nms
creating content/gdrive/My Drive/ML/planercnn/nms/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/in

In [ ]:
!pip install torchvision==0.2.2
!pip install torch==0.4.1

# Display MidasNet

In [5]:
# !brew install graphviz
# !pip3 install torchviz

# from torchviz import make_dot, make_dot_from_trace

import torch

from torchvision.transforms import Compose
from MiDaS.midas.midas_net import MidasNet
from MiDaS.midas.transforms import Resize, NormalizeImage, PrepareForNet
from torchsummary import summary

MIDAS_MODEL_PATH = "/content/gdrive/My Drive/ML/model/midas-f46da743.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: %s" % device)

# load network
midas_model = MidasNet(MIDAS_MODEL_PATH, non_negative=True)
midas_model.to(device)
#print(model.children())
print("MidasNet: ")

summary(midas_model, (3,224, 224))
child_counter = 0
for name, child in midas_model.named_children():
    print(" child", name, "is -")
    print(child)
    child_counter += 1

# x = torch.randn(32,3,224, 224).to(device).requires_grad_(True)
# y = midas_model(x)
# make_dot((y[0],y[1]), params=dict(list(midas_model.named_parameters())))


device: cuda
Loading weights:  /content/gdrive/My Drive/ML/model/midas-f46da743.pt


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


MidasNet: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 256, 56, 56]          16,384
       BatchNorm2d-6          [-1, 256, 56, 56]             512
              ReLU-7          [-1, 256, 56, 56]               0
            Conv2d-8          [-1, 256, 56, 56]          18,432
       BatchNorm2d-9          [-1, 256, 56, 56]             512
             ReLU-10          [-1, 256, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          65,536
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14        

# Display Yolo V3

In [6]:
# !brew install graphviz
# !pip3 install torchviz

# from torchviz import make_dot, make_dot_from_trace

import torch

from torchvision.transforms import Compose
from torchsummary import summary

YOLOV3_MODEL_PATH = "/content/gdrive/My Drive/ML/model/yolov3.pt"
YOLOV3_CONFIG_PATH =  "/content/gdrive/My Drive/ML/YoloV3/cfg/yolov3-custom.cfg"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: %s" % device)

from YoloV3.models import *  # set ONNX_EXPORT in models.py
from YoloV3.utils.datasets import *
from YoloV3.utils.utils import *

img_size = (3, 224, 224)
yolov3_model = Darknet(YOLOV3_CONFIG_PATH, img_size)

yolov3_model.to(device).requires_grad_(True)

# Load weights
attempt_download(YOLOV3_MODEL_PATH)
if YOLOV3_MODEL_PATH.endswith('.pt'):  # pytorch format
    yolov3_model.load_state_dict(torch.load(YOLOV3_MODEL_PATH, map_location=device)['model'])
else:  # darknet format
    load_darknet_weights(yolov3_model, YOLOV3_MODEL_PATH)

print("Yolo V3: ")

# summary(yolov3_model, (3,224,224))

child_counter = 0
for name, child in yolov3_model.named_children():
    print("YoloV3 child", child_counter, "is -")
    print("name = ", name, "child = ", child)
    child_counter += 1

# x = torch.randn(32,3,224, 224).to(device).requires_grad_(True)
# y = yolov3_model(x)
# make_dot((y[0],y[1]), params=dict(list(yolov3_model.named_parameters())))

# print (yolov3_model.named_children())

# for name, module in yolov3_model.named_children():
#   print("name = ", name, "Module = ", module)


## To be uncommented when trying out PlaneRCNN
# from planercnn.models.model import *
#from planercnn.models.refinement_net import RefineModel
#from planercnn.models.modules import *

# planer_model = MaskRCNN(config)
# planer_model.cuda()

# child_counter = 0
# for child in planer_model.children():
#     print("PlaneRCNN child", child_counter, "is -")
#     print(child)
#     child_counter += 1



device: cuda
Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
Yolo V3: 
YoloV3 child 0 is -
name =  module_list child =  ModuleList(
  (0): Sequential(
    (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (1): Sequential(
    (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (2): Sequential(
    (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
    (activation): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (3): Sequential(
    (Co

# Tricycle Model created and Weights Loaded from Base Models & then Saved Weights of the new Tricycle Model for later use

In [8]:
!pwd

import torch

from torchvision.transforms import Compose
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: %s" % device)


from Tricycle.models import *  # set ONNX_EXPORT in models.py
from Tricycle.utils.datasets import *
from Tricycle.utils.utils import *

TRICYCLE_CONFIG_PATH =  "/content/gdrive/My Drive/ML/Tricycle/cfg/tricycle.cfg"
MODEL_FILE = "/content/gdrive/My Drive/ML/Tricycle/checkpoint/last.pt"

img_size = (416, 416)
tricycle_model = Darknet(TRICYCLE_CONFIG_PATH, img_size, verbose = False)

tricycle_model.to(device)

print("Tricycle: ")

summary(tricycle_model, (3, 416,416))

child_counter = 0
for name, child in tricycle_model.named_children():
    print("Tricycle child", child_counter, "is -")
    print("name = ", name, "child = ", child)
    child_counter += 1

tricycle_model.load_weights(midas_model, yolov3_model)


# print (tricycle_model.module_list[0][1].weight.data)


chkpt = {'epoch': 0, 
        'best_fitness': 0, 
        'training_results': "", 
        'model': tricycle_model.module.state_dict() if hasattr(tricycle_model, 'module') else tricycle_model.state_dict(), 
        'optimizer': None} 

# Save last, best and delete 
# output = open(MODEL_FILE, mode="wb")
torch.save(chkpt, MODEL_FILE) 



/content/gdrive/My Drive/ML
device: cuda


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


Model Summary: 421 layers, 1.22123e+08 parameters, 1.22123e+08 gradients
Tricycle: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 208, 208]           9,408
       BatchNorm2d-2         [-1, 64, 208, 208]             128
              ReLU-3         [-1, 64, 208, 208]               0
         MaxPool2d-4         [-1, 64, 104, 104]               0
            Conv2d-5        [-1, 256, 104, 104]          16,384
       BatchNorm2d-6        [-1, 256, 104, 104]             512
              ReLU-7        [-1, 256, 104, 104]               0
            Conv2d-8        [-1, 256, 104, 104]          18,432
       BatchNorm2d-9        [-1, 256, 104, 104]             512
             ReLU-10        [-1, 256, 104, 104]               0
           Conv2d-11        [-1, 256, 104, 104]          65,536
      BatchNorm2d-12        [-1, 256, 104, 104]             512
           Conv2d-1

# Attempt 1 & 2 for dynamically created model:

In [ ]:
!ls

from tricycle_net import TricycleNet

trimodel = TricycleNet()
trimodel.to(device)
#summary(trimodel, (3,224, 224))
# summary(trimodel, (3, 224,224))

child_counter = 0
for name, child in trimodel.named_children():
    print("name = ", name, "child = ", child)
    child_counter += 1



data  MiDaS  model  planercnn  __pycache__  tricycle_net.py  YoloV3
Loading weights:  None
device: cuda
Loading weights:  /content/gdrive/My Drive/ML/model/midas-f46da743.pt


Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master


Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients
name =  midas_model child =  MidasNet(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNo